This notebook draws out the trees for European and American options.

The first half of the code uses sigma and continuous compounding of the risk-free rate.

The second half of the code uses u/d and discrete discounting.

### Keep T and n the same for simplicity sake

In [2]:
import math
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
import plotly.graph_objects as go
import pandas as pd
import datetime as dt
import warnings

In [3]:
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

%matplotlib ipympl

# Using sigma

In [3]:
# Example usage
S = 100  # initial stock price
K = 100  # strike price
r = 0.1  # risk-free interest rate
sigma = 0.4  # stock price volatility
T = 4  # time to maturity (in years)
N = 4  # number of time steps in the binomial tree

### Eurpoean Calls and Puts

In [4]:
def binomial_tree_european_call(S, K, r, sigma, T, N):
    """
    Calculates the price of a European call option using a multi-step binomial tree.

    Parameters:
        S (float): initial stock price
        K (float): strike price
        r (float): risk-free interest rate
        sigma (float): stock price volatility
        T (float): time to maturity (in years)
        N (int): number of time steps in the binomial tree

    Returns:
        (float) price of the call option
    """
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u                             # Cox-Ross-Rubinstein formulation
    p = (np.exp(r * dt) - d) / (u - d)    # Risk neutral probability of going up, this needs to be between 0 and 1

    # Generate the stock price tree
    stock_tree = np.zeros((N+1, N+1))
    for i in range(N+1):                  # Loops through the columns
        for j in range(i+1):              # Loop through the rows
            stock_tree[j, i] = S * (u ** (i-j)) * (d ** j)    # Initial stock price multiplied by up amount (power of number of times it went up) multiplied by down amount (power of number of time it went down)

    # Print the stock price tree
    print("Stock price tree:")
    print(stock_tree)

    # Generate the option value tree
    option_tree = np.zeros((N+1, N+1))
    option_tree[:, N] = np.maximum(stock_tree[:, N] - K, 0)       # Payoff at maturity
    for i in range(N-1, -1, -1):          # Loops through the columns
        for j in range(i+1):              # Loop through the rows
            option_tree[j, i] = np.exp(-r * dt) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])
            #option_tree[j, i] = max(option_tree[j, i], stock_tree[j, i] - K, 0)      # Do we need to add in a 0 here?

    # Print the option price tree
    print("Option price tree:")
    print(option_tree)

    return option_tree[0, 0]

In [5]:
def binomial_tree_european_put(S, K, r, sigma, T, N):
    """
    Calculates the price of a European put option using a multi-step binomial tree.

    Parameters:
        S (float): initial stock price
        K (float): strike price
        r (float): risk-free interest rate
        sigma (float): stock price volatility
        T (float): time to maturity (in years)
        N (int): number of time steps in the binomial tree

    Returns:
        (float) price of the put option
    """
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u                             # Cox-Ross-Rubinstein formulation
    p = (np.exp(r * dt) - d) / (u - d)    # Risk neutral probability of going up, this needs to be between 0 and 1

    # Generate the stock price tree
    stock_tree = np.zeros((N+1, N+1))
    for i in range(N+1):                  # Loops through the columns
        for j in range(i+1):              # Loop through the rows
            stock_tree[j, i] = S * (u ** (i-j)) * (d ** j)    # Initial stock price multiplied by up amount (power of number of times it went up) multiplied by down amount (power of number of time it went down)

    # Print the stock price tree
    print("Stock price tree:")
    print(stock_tree)

    # Generate the option value tree
    option_tree = np.zeros((N+1, N+1))
    option_tree[:, N] = np.maximum(K - stock_tree[:, N], 0)    # Payoff at maturity
    for i in range(N-1, -1, -1):          # Loops through the columns
        for j in range(i+1):              # Loop through the rows
            option_tree[j, i] = np.exp(-r * dt) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])
            #option_tree[j, i] = max(option_tree[j, i], K - stock_tree[j, i], 0)    # Do we need to add in a 0 here?

    # Print the option price tree
    print("Option price tree:")
    print(option_tree)

    return option_tree[0, 0]

### American Calls and Puts

In [6]:
def binomial_tree_american_call(S, K, r, sigma, T, N):
    """
    Calculates the price of an American call option using a multi-step binomial tree.

    Parameters:
        S (float): initial stock price
        K (float): strike price
        r (float): risk-free interest rate
        sigma (float): stock price volatility
        T (float): time to maturity (in years)
        N (int): number of time steps in the binomial tree

    Returns:
        (float) price of the call option
    """
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u                              # Cox-Ross-Rubinstein formulation
    p = (np.exp(r * dt) - d) / (u - d)     # Risk neutral probability of going up, this needs to be between 0 and 1

    # Generate the stock price tree
    stock_tree = np.zeros((N+1, N+1))
    for i in range(N+1):                   # Loops through the columns
        for j in range(i+1):               # Loops through the rows
            stock_tree[j, i] = S * (u ** (i-j)) * (d ** j)   # Initial stock price multiplied by up amount (power of number of times it went up) multiplied by down amount (power of number of time it went down)

    # Print the stock price tree
    print("Stock price tree:")
    print(stock_tree)
    
    # Generate the option value tree
    option_tree = np.zeros((N+1, N+1))
    option_tree[:, N] = np.maximum(stock_tree[:, N] - K, 0)
    for i in range(N-1, -1, -1):           # Loops through the columns
        for j in range(i+1):               # Loops through the rows
            exercise_value = stock_tree[j, i] - K
            hold_value = np.exp(-r * dt) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])
            option_tree[j, i] = max(exercise_value, hold_value, 0)

    # Print the option price tree
    print("Option price tree:")
    print(option_tree)
    
    return option_tree[0, 0]

In [7]:
def binomial_tree_american_put(S, K, r, sigma, T, N):
    """
    Calculates the price of an American put option using a multi-step binomial tree.

    Parameters:
        S (float): initial stock price
        K (float): strike price
        r (float): risk-free interest rate
        sigma (float): stock price volatility
        T (float): time to maturity (in years)
        N (int): number of time steps in the binomial tree

    Returns:
        (float) price of the put option
    """
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u                              # Cox-Ross-Rubinstein formulation
    p = (np.exp(r * dt) - d) / (u - d)     # Risk neutral probability of going up, this needs to be between 0 and 1

    # Generate the stock price tree
    stock_tree = np.zeros((N+1, N+1))
    for i in range(N+1):                   # Loops through the columns
        for j in range(i+1):               # Loops through the rows
            stock_tree[j, i] = S * (u ** (i-j)) * (d ** j)   # Initial stock price multiplied by up amount (power of number of times it went up) multiplied by down amount (power of number of time it went down)

    # Print the stock price tree
    print("Stock price tree:")
    print(stock_tree)

    # Generate the option value tree
    option_tree = np.zeros((N+1, N+1))
    option_tree[:, N] = np.maximum(K - stock_tree[:, N], 0)
    for i in range(N-1, -1, -1):          # Loops through the columns
        for j in range(i+1):              # Loops through the rows
            exercise_value = K - stock_tree[j, i]
            hold_value = np.exp(-r * dt) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])
            option_tree[j, i] = max(exercise_value, hold_value, 0)        # Do we need to add in a 0 here?

    # Print the option price tree
    print("Option price tree:")
    print(option_tree)

    return option_tree[0, 0]

In [8]:
european_call_price = binomial_tree_european_call(S, K, r, sigma, T, N)
print("European Call price:", european_call_price)

Stock price tree:
[[100.         149.18246976 222.55409285 332.01169227 495.30324244]
 [  0.          67.0320046  100.         149.18246976 222.55409285]
 [  0.           0.          44.93289641  67.0320046  100.        ]
 [  0.           0.           0.          30.11942119  44.93289641]
 [  0.           0.           0.           0.          20.1896518 ]]
Option price tree:
[[4.37423105e+01 7.93540904e+01 1.40681018e+02 2.41527950e+02
  3.95303242e+02]
 [0.00000000e+00 1.34657464e+01 2.81144479e+01 5.86987280e+01
  1.22554093e+02]
 [0.00000000e+00 0.00000000e+00 3.26003258e-15 6.80645644e-15
  1.42108547e-14]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]]
European Call price: 43.74231053246738


In [9]:
european_put_price = binomial_tree_european_put(S, K, r, sigma, T, N)
print("European Put price:", european_put_price)

Stock price tree:
[[100.         149.18246976 222.55409285 332.01169227 495.30324244]
 [  0.          67.0320046  100.         149.18246976 222.55409285]
 [  0.           0.          44.93289641  67.0320046  100.        ]
 [  0.           0.           0.          30.11942119  44.93289641]
 [  0.           0.           0.           0.          20.1896518 ]]
Option price tree:
[[10.77431514  4.25344272  0.          0.          0.        ]
 [ 0.         20.51556384  9.98752326  0.          0.        ]
 [ 0.          0.         36.9401789  23.4517372   0.        ]
 [ 0.          0.          0.         60.36432061 55.06710359]
 [ 0.          0.          0.          0.         79.8103482 ]]
European Put price: 10.77431513603128


In [10]:
american_call_price = binomial_tree_american_call(S, K, r, sigma, T, N)
print("American Call price:", american_call_price)

Stock price tree:
[[100.         149.18246976 222.55409285 332.01169227 495.30324244]
 [  0.          67.0320046  100.         149.18246976 222.55409285]
 [  0.           0.          44.93289641  67.0320046  100.        ]
 [  0.           0.           0.          30.11942119  44.93289641]
 [  0.           0.           0.           0.          20.1896518 ]]
Option price tree:
[[4.37423105e+01 7.93540904e+01 1.40681018e+02 2.41527950e+02
  3.95303242e+02]
 [0.00000000e+00 1.34657464e+01 2.81144479e+01 5.86987280e+01
  1.22554093e+02]
 [0.00000000e+00 0.00000000e+00 3.26003258e-15 6.80645644e-15
  1.42108547e-14]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]]
American Call price: 43.74231053246738


In [11]:
american_put_price = binomial_tree_american_put(S, K, r, sigma, T, N)
print("American Put price:", american_put_price)

Stock price tree:
[[100.         149.18246976 222.55409285 332.01169227 495.30324244]
 [  0.          67.0320046  100.         149.18246976 222.55409285]
 [  0.           0.          44.93289641  67.0320046  100.        ]
 [  0.           0.           0.          30.11942119  44.93289641]
 [  0.           0.           0.           0.          20.1896518 ]]
Option price tree:
[[16.90417313  5.97940693  0.          0.          0.        ]
 [ 0.         32.9679954  14.04026567  0.          0.        ]
 [ 0.          0.         55.06710359 32.9679954   0.        ]
 [ 0.          0.          0.         69.88057881 55.06710359]
 [ 0.          0.          0.          0.         79.8103482 ]]
American Put price: 16.904173128843627


# Using u/d

In [10]:
# Define the parameters of the binomial tree
S = 5    # Initial stock price
K = 10    # Strike price
u = 2      # Up factor
d = 0.5      # Down factor
r = 0.04   # Risk-free rate
T = 2      # Time to maturity in years
N = 2      # Number of time steps

### Basic European Calls and Puts

In [11]:
def binomial_tree_basic_european_call(S, K, r, u, d, T, N):
    """
    Calculates the price of a European call option using a multi-step binomial tree.

    Parameters:
        S (float): initial stock price
        K (float): strike price
        r (float): risk-free interest rate
        u: Up factor
        d: Down factor
        T (float): time to maturity (in years)
        N (int): number of time steps in the binomial tree

    Returns:
        (float) price of the put option
    """
    dt=T/N
    # Compute the risk-neutral probabilities of moving up and down
    p = (((1+r)**dt) - d) / (u - d)          # Check this
    q = 1 - p
    
    print(p)
    print(q)

    # Generate the stock price tree
    stock_tree = np.zeros((N+1, N+1))
    for i in range(N+1):                  # Loops through the columns
        for j in range(i+1):              # Loop through the rows
            stock_tree[j, i] = S * (u ** (i-j)) * (d ** j)    # Initial stock price multiplied by up amount (power of number of times it went up) multiplied by down amount (power of number of time it went down)

    # Print the stock price tree
    print("Stock price tree:")
    print(stock_tree)

    # Generate the option value tree
    option_tree = np.zeros((N+1, N+1))
    option_tree[:, N] = np.maximum(stock_tree[:, N] - K, 0)    # Payoff at maturity
    for i in range(N-1, -1, -1):          # Loops through the columns
        for j in range(i+1):              # Loop through the rows
            option_tree[j, i] = (1/((1+r)**dt)) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])    # Check this
            #option_tree[j, i] = max(option_tree[j, i], K - stock_tree[j, i], 0)    # Do we need to add in a 0 here?

    # Print the option price tree
    print("Option price tree:")
    print(option_tree)

    return option_tree[0, 0]

In [12]:
def binomial_tree_basic_european_put(S, K, r, u, d, T, N):
    """
    Calculates the price of a European put option using a multi-step binomial tree.

    Parameters:
        S (float): initial stock price
        K (float): strike price
        r (float): risk-free interest rate
        u: Up factor
        d: Down factor
        T (float): time to maturity (in years)
        N (int): number of time steps in the binomial tree

    Returns:
        (float) price of the put option
    """
    dt=T/N
    # Compute the risk-neutral probabilities of moving up and down
    p = (((1+r)**dt) - d) / (u - d)         # Check this
    q = 1 - p
    
    print(p)
    print(q)

    # Generate the stock price tree
    stock_tree = np.zeros((N+1, N+1))
    for i in range(N+1):                  # Loops through the columns
        for j in range(i+1):              # Loop through the rows
            stock_tree[j, i] = S * (u ** (i-j)) * (d ** j)    # Initial stock price multiplied by up amount (power of number of times it went up) multiplied by down amount (power of number of time it went down)

    # Print the stock price tree
    print("Stock price tree:")
    print(stock_tree)

    # Generate the option value tree
    option_tree = np.zeros((N+1, N+1))
    option_tree[:, N] = np.maximum(K - stock_tree[:, N], 0)    # Payoff at maturity
    for i in range(N-1, -1, -1):          # Loops through the columns
        for j in range(i+1):              # Loop through the rows
            option_tree[j, i] = (1/((1+r)**dt)) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])    # Check this
            #option_tree[j, i] = max(option_tree[j, i], K - stock_tree[j, i], 0)    # Do we need to add in a 0 here?

    # Print the option price tree
    print("Option price tree:")
    print(option_tree)

    return option_tree[0, 0]

### Basic American Calls and Puts

In [13]:
def binomial_tree_basic_american_call(S, K, r, u, d, T, N):
    """
    Calculates the price of an American call option using a multi-step binomial tree.

    Parameters:
        S (float): initial stock price
        K (float): strike price
        r (float): risk-free interest rate
        u: Up factor
        d: Down factor
        T (float): time to maturity (in years)
        N (int): number of time steps in the binomial tree

    Returns:
        (float) price of the put option
    """
    dt=T/N
    # Compute the risk-neutral probabilities of moving up and down
    p = (((1+r)**dt) - d) / (u - d)      # Check this
    q = 1 - p
    
    print(p)
    print(q)
    
    # Generate the stock price tree
    stock_tree = np.zeros((N+1, N+1))
    for i in range(N+1):                   # Loops through the columns
        for j in range(i+1):               # Loops through the rows
            stock_tree[j, i] = S * (u ** (i-j)) * (d ** j)   # Initial stock price multiplied by up amount (power of number of times it went up) multiplied by down amount (power of number of time it went down)

    # Print the stock price tree
    print("Stock price tree:")
    print(stock_tree)

    # Generate the option value tree
    option_tree = np.zeros((N+1, N+1))
    option_tree[:, N] = np.maximum(stock_tree[:, N] - K, 0)
    for i in range(N-1, -1, -1):          # Loops through the columns
        for j in range(i+1):              # Loops through the rows
            exercise_value = stock_tree[j, i] - K
            hold_value = (1/((1+r)** dt)) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])   # Check this
            option_tree[j, i] = max(exercise_value, hold_value, 0)        # Do we need to add in a 0 here?

    # Print the option price tree
    print("Option price tree:")
    print(option_tree)

    return option_tree[0, 0]

In [14]:
def binomial_tree_basic_american_put(S, K, r, u, d, T, N):
    """
    Calculates the price of an American put option using a multi-step binomial tree.

    Parameters:
        S (float): initial stock price
        K (float): strike price
        r (float): risk-free interest rate
        u: Up factor
        d: Down factor
        T (float): time to maturity (in years)
        N (int): number of time steps in the binomial tree

    Returns:
        (float) price of the put option
    """
    dt=T/N
    # Compute the risk-neutral probabilities of moving up and down
    p = (((1+r)**dt) - d) / (u - d)          # Check this
    q = 1 - p
    
    print(p)
    print(q)
    
    # Generate the stock price tree
    stock_tree = np.zeros((N+1, N+1))
    for i in range(N+1):                   # Loops through the columns
        for j in range(i+1):               # Loops through the rows
            stock_tree[j, i] = S * (u ** (i-j)) * (d ** j)   # Initial stock price multiplied by up amount (power of number of times it went up) multiplied by down amount (power of number of time it went down)

    # Print the stock price tree
    print("Stock price tree:")
    print(stock_tree)

    # Generate the option value tree
    option_tree = np.zeros((N+1, N+1))
    option_tree[:, N] = np.maximum(K - stock_tree[:, N], 0)
    for i in range(N-1, -1, -1):          # Loops through the columns
        for j in range(i+1):              # Loops through the rows
            exercise_value = K - stock_tree[j, i]
            hold_value = (1/((1+r)** dt)) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])   # Check this
            option_tree[j, i] = max(exercise_value, hold_value, 0)        # Do we need to add in a 0 here?

    # Print the option price tree
    print("Option price tree:")
    print(option_tree)

    return option_tree[0, 0]

In [15]:
basic_european_call_price = binomial_tree_basic_european_call(S, K, r, u, d, T, N)
print("European Call price:", basic_european_call_price)

0.36000000000000004
0.6399999999999999
Stock price tree:
[[ 5.   10.   20.  ]
 [ 0.    2.5   5.  ]
 [ 0.    0.    1.25]]
Option price tree:
[[ 1.19822485  3.46153846 10.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
European Call price: 1.198224852071006


In [19]:
basic_european_put_price = binomial_tree_basic_european_put(S=5, K=10, r=0.04, u=2, d=0.5, T=2, N=2)
print("European Put price:", basic_european_put_price)

0.36000000000000004
0.6399999999999999
Stock price tree:
[[ 5.   10.   20.  ]
 [ 0.    2.5   5.  ]
 [ 0.    0.    1.25]]
Option price tree:
[[5.44378698 3.07692308 0.        ]
 [0.         7.11538462 5.        ]
 [0.         0.         8.75      ]]
European Put price: 5.443786982248518


In [17]:
basic_american_call_price = binomial_tree_basic_american_call(S, K, r, u, d, T, N)
print("American Call price:", basic_american_call_price)

0.36000000000000004
0.6399999999999999
Stock price tree:
[[ 5.   10.   20.  ]
 [ 0.    2.5   5.  ]
 [ 0.    0.    1.25]]
Option price tree:
[[ 1.19822485  3.46153846 10.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
American Call price: 1.198224852071006


In [18]:
basic_american_put_price = binomial_tree_basic_american_put(S, K, r, u, d, T, N)
print("American Put price:", basic_american_put_price)

0.36000000000000004
0.6399999999999999
Stock price tree:
[[ 5.   10.   20.  ]
 [ 0.    2.5   5.  ]
 [ 0.    0.    1.25]]
Option price tree:
[[5.68047337 3.07692308 0.        ]
 [0.         7.5        5.        ]
 [0.         0.         8.75      ]]
American Put price: 5.680473372781063
